### Infêrência Estatística
# Título: Avaliação do impacto da expressões multipalavras nos modelos de linguagem

Este script é similar ao caso base. Mas antes de criar o modelo de linguagem, identificamos as possíveis expressões multipalavras e substituímos a expressão por um único token.

In [26]:
import markovify
import os
import time
import json
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import MWETokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
import math
import numpy as np

Baseamos o código no Read me da biblioteca "Markovify" em: https://github.com/jsvine/markovify/blob/master/README.md

Primeiro criamos uma função para preprocessar o texto.

In [27]:
# Primeiro, criamos uma função para preprocessar os textos
def preprocess(text):
    # Decodificar usando ISO 8859-1 
    text = text.decode('ISO 8859-1')
    # Colocar todas as letras em minúscula
    text = text.lower()
    # Separar as sentenças pelos pontos de finais
    text = text.split('.')
    new_text = []
    for sent in text:
        # Eliminamos as sentenças com menos de 100 caracteres
        if len(sent)> 100:
            # Excluir os espaços no início e fim das sentenças
            new_text.append(sent.strip())
    return(new_text)

Unir os textos em um corpus único para identificar a frequência das palavras e dos bigrams

In [48]:
# Contador de tempo
start_time = time.time()

# Dicionário para armazenar a frequência da palavras
freq_words = {}
freq_bigram = {}

for (dirpath, _, filenames) in os.walk("Originais/treino"):
    for filename in filenames:

        
        # Ler o arquivo e recuperar a frequência das palavras e dos bibrams
        with open(os.path.join(dirpath, filename), 'rb') as f:
            # Preprocessar o texto lido
            text = preprocess(f.read())
            # Iterando por cada sentença
            for sent in text:
                # tokenizando as sentenças
                sent = nltk.word_tokenize(sent)
                # Recuperando a frequência das palavras da sentença
                freq = FreqDist(sent)
                # Atualizando o dicionário com frequência das palavras
                for i in freq:
                    try:
                        freq_words[i] = freq_words[i] + freq[i]
                    except:
                        freq_words[i] = freq[i]
                # Recuperando os bigram
                bigram = nltk.bigrams(sent)
                for i in bigram:
                    try:
                        freq_bigram[i] = freq_bigram[i] + 1
                    except:
                        freq_bigram[i] = 1      
                        
# Fim do contador de tempo
print ("--- %s seconds ---" % (time.time() - start_time))

--- 48.863097190856934 seconds ---


Com a informação das frequências das palavras e dos bigram, é possível calcular a informação mútua.

In [49]:
# Calculando a Informação Mútua para cada bigram
bigram_MI = {}
n_bigram = len(freq_bigram)
n_word = len(freq_words)
for bi in freq_bigram:
    bigram_MI[bi] = math.log((freq_bigram[bi]/n_bigram)/
                             ((freq_words[bi[0]]/n_bigram)*(freq_words[bi[1]]/n_bigram)))

In [50]:
# Imprimindo o número total de palavras encontrado
sum(freq_words.values())


3681623

In [51]:
# Imprimindo o número total de bigrams encontrado
print (len(bigram_MI))

722623


Vamos considerar que um bigram é uma MWE aqueles pares de palavras cuja Informação Mútua ultrapassar um determinado limite.

In [32]:
# Criar uma lista de MWE com os bigrams que ultrapassem o limite assinalado na variável "MI".
mwe = {}
MI = 12.0
for bi in bigram_MI:
    if bigram_MI[bi] > MI:
        mwe[bi] = bigram_MI[bi]
print(len(mwe))
for e in mwe:
    print(bigram_MI[e], e[0], e[1])

3852
12.79749574613375  p o l i a m i n a   c i c l o a l i f á t i c a 
13.490642926693695  p ' -   d i h i d r o x i d i f e n i l d i m e t i l m e t a n o 
12.392030638025586  s c i f i d e r   s c h o l a r 
12.79749574613375  h a l o g ê n i o   e h j 
13.490642926693695  b a t o n   r o u g e 
12.79749574613375  s w i n g   a d s o r p t i o n 
13.490642926693695  t g a / s d t a   8 5 1 e 
12.392030638025586  t p d / t p r   2 9 0 0 
12.392030638025586  f u n d e n t e   l e c o c e l 
13.490642926693695  t s - f f   - a a s 
12.79749574613375  b o o s t e d - d i s c h a r g e   h o l l o w 
12.79749574613375  h o l l o w   c a t h o d e 
12.79749574613375  a y a m e   e x t r a í r a m 
13.490642926693695  6 - d i m e t i l   - 4 - h e p t a n o 
13.490642926693695  d u o l i t e   g t - 7 3 
13.490642926693695  8 - h y d r o x i   q u i n o l i n a 
12.104348565573805  a l i t e a   c - 6 
12.79749574613375  h i p e r e s f e r a   e q ü i d i s t a n t e 
12.104348565573805

13.490642926693695  a d e s   e s p a c i a 
13.490642926693695  m é t   o r r e a t o r e s 
13.490642926693695  t é r m i   u r a s 
13.490642926693695  a n ó d   e n t a d o 
13.490642926693695  a l u   g a n l e y 
13.490642926693695  m o r f o l   u m i n a 
13.490642926693695  i c r o r r e a t o r e   a r t i r 
13.490642926693695  c e r â m   o m p o s i ç ã o 
13.490642926693695  m i c r o r   n s i d a d e 
13.490642926693695  m i c r o c a   m i c r o m a n u f a 
12.392030638025586  a n d o   d i v e r s 
13.490642926693695  c o - r e / gð- m b é m   r e c o b e 
13.490642926693695  r e c o b e   c u l a r e s 
13.490642926693695  c u l a r e s   t a m b 
13.490642926693695  t a m b   n c i a 
13.490642926693695  s u p e r f í c i   e t r o 
13.490642926693695  m i c r o m o n ó   ó l i t o 
13.490642926693695  r f í c i e   d i s p o n 
13.490642926693695  r e c o b   o s c o p i a 
13.490642926693695  o s c o p i a   e l e t 
13.490642926693695  m o s t r   t r u t u r a 

12.392030638025586  p u e d a   s e g u i r l e s 
12.79749574613375  f r e c u e n t e m e n t e   s u c e d e r á 
13.490642926693695  s u y o s   p r o p i o s 
13.490642926693695  h a c e r l o  1 4 0 
13.490642926693695  s e g ú n   é s t o 
13.490642926693695  e x a l t a c i ó n  1 4 1 
12.79749574613375  e s c a p a r a t e  1 4 2 
13.490642926693695  i n t e n t ó   s i l e n c i a r 
12.392030638025586  h a c i a   a c o s t a r 
13.490642926693695  f a s t u o s o s   d e s f i l e s 
12.392030638025586  s i g u e   r e p i t i e n d o 
12.392030638025586  s e r á n   c a s i 
13.490642926693695  c a s i   d i e z 
13.490642926693695  a c o n t e c i m i e n t o s   t e r r o r í f i c o s 
12.104348565573805  n u e v o s   d e s a p a r e c i d o s 
12.79749574613375  d e s a p a r e c i d o s  1 4 5 
12.79749574613375  t r i u n f o  1 4 6 
12.79749574613375 t ú   v i v i a s 
12.104348565573805  c a l l e   m e t i d a 
12.79749574613375  q u i e n e s   c o n o c i a s 


13.490642926693695  v e i u   f a l l a r 
13.490642926693695  c o m m u n i c a n d o - m e   v e r b a l m e n t e 
12.79749574613375  f ô r a   c o m i d o 
12.79749574613375  i b é r i c o   r e p e r c u t i u 
12.104348565573805  c a r t o g r á f i c o   i n t a c t o 
13.490642926693695  o n ç a   p i n t a d a 
12.392030638025586  t o p o n í m i a   n e e r l a n d e s a 
13.490642926693695  b a r l é u   a n o t a 
12.79749574613375  e x p e d i c i o n á r i o s   e n c o r a j a v a m 
13.490642926693695  v i e j o s   í d o l o s 
12.392030638025586  í n d i o  - a o 
12.104348565573805  à r e a s   p i o n e i r a s 
12.392030638025586  i n c u t i r   c i v i l i d a d e 
12.104348565573805  m e s q u i t a   p i m e n t e l 
12.392030638025586 s u i   g e n e r i s 
12.392030638025586  1 9 6 0 / 1 9 7 0   c o n s t i t u í r a m - s e 
12.79749574613375  e d m u n d o   o r l a n d i n i 
12.79749574613375  t a q u a r i   v a s s o u r a s 
12.392030638025586  d i v i

12.79749574613375  o n i o n   r o o t - t i p s 
13.490642926693695  u n c a r i a   t o m e n t o s a 
12.392030638025586  h y d r a t e d   s u l f u r 
12.104348565573805  m i c r o s o m a l   e n z y m e s 
13.490642926693695  f l u o r o c r o m o s   b a s e - e s p e c í f i c o s 
13.490642926693695  t é n i c a s   c i t o g e n é t i c a s 
12.392030638025586  a n e m i a   h e m o l í t i c a 
13.490642926693695  7  - d i c l o r o f l u o r e s c e í n a 
12.79749574613375 - d i c l o r o f l u o r e s c e í n a   d i a c e t a t o 
12.104348565573805 s  s 
12.104348565573805 t y r  t y r 
12.104348565573805  g s t s   c i t o s ó l i c a s 
12.79749574613375  g u t a t i o n a   s - t r a n f e r a s e 
13.490642926693695  8 - h i d r o x i - 2  - d e s o g u a n o s i n a 
13.490642926693695  d i p h e n y l t e t r a z o l i u m   b r o m i d e 
12.79749574613375  c i c l o d i e n o s   a l d r i n 
13.490642926693695  p  - d d t 
12.392030638025586  4 - á c i 

12.392030638025586  f e t o   d e r m i c o 
13.490642926693695  p r o t o z o á r i o   c i l i a d o 
12.104348565573805  a l f o n i c   8 1 0 - 6 0 
12.104348565573805  p o l y c h l o r i d e   b i p h e n y l 
12.79749574613375  s a p o g e n a t   t - 3 0 0 
13.490642926693695  a k o p a l   n - 3 0 0 
13.490642926693695  o b s e r e v a ç õ e s   d e g r a ç ã o 
12.392030638025586  l i n e r   d e s c a r t á v e l 
13.490642926693695  s s - 0 3   s s - 0 4 
13.490642926693695  s s - 0 4   s s - 0 5 
13.490642926693695  s s - 0 1 / 4   s s - 0 3 / 1 
13.490642926693695  s s - 0 3 / 1   s s - 0 4 / 2 
13.490642926693695  s s - 0 4 / 2   s s - 0 5 / 3 


In [33]:
# Função que recebe um sentença e o dicionário com MWE e retorna a sentença com os MWE unidos por "_". 

def uni_MWE(sent, mwe):
    
    mwe_tokenizer = MWETokenizer(list(mwe.keys()))
    sent = nltk.word_tokenize(sent)
    sent = mwe_tokenizer.tokenize(sent)
    sent = TreebankWordDetokenizer().detokenize(sent)
    return (sent)

Para cada arquivos do diretórios especificado lemos e preprocessamos o texto, unimos os bigrams considerados MWE, criamos uma rede de markov com os n-grams (state_size) e combinamos os modelos de todos os arquivos.

In [34]:
# Contador de tempo
start_time = time.time()

# Cadeia de Markov combinada
combined_model = None

# Contagem de arquivos
n_files = 0

# Iterar pelos diretórios e arquivos
for (dirpath, _, filenames) in os.walk("Originais/treino"):
    for filename in filenames:
        n_files += 1
        
        try:
            # Ler o arquivo e criar uma cadeia de Markov para cada arquivo
            with open(os.path.join(dirpath, filename), 'rb') as f:
                
                # Preprocessar o texto lido
                text = preprocess(f.read())
                text_mwe = []
                
                # Unir as MWE
                for sent in text:
                    text_mwe.append(uni_MWE(sent, mwe))
                
                # Treinar o modelos usando as o texto preprocessado e com MWE
                model = markovify.NewlineText(text_mwe, retain_original=False, state_size=2)

                # Combinar modelo criado para um arquivo com a Cadeia de Markov combinada
                if combined_model:
                    combined_model = markovify.combine(models=[combined_model, model])
                else:
                    combined_model = model
        except:
            print("Não foi possível ler o arquivo", filename)
            #pass
                    
        # Contador de arquivos processados
        if n_files % 50 == 0:
            print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))
    
# Fim do contador de tempo
print ("--- %s seconds ---" % (time.time() - start_time))

--- 50 arquivos processados --- --- 537.3928475379944 seconds ---
--- 100 arquivos processados --- --- 1143.8330619335175 seconds ---
--- 1596.6718480587006 seconds ---


In [35]:
# Imprimir um exemplo de sentença aleatória
print(combined_model.make_sentence(max_words=50))

 1 1 ,    r e s p e c t i v a m e n t e 


In [36]:
# Gravar modelo em JSON em disco
#model_json = combined_model.to_json()
#with open('MarkovChainModel_mwe_MI12.json', 'w') as f:
#    json.dump(model_json, f)

In [37]:
# Carregar modelo em JSON do disco
#with open('MarkovChainModel_mwe_MI12.json', 'r') as f:
#    model_json = markovify.Text.from_json(json.load(f))

Como os textos usados são em português, foi necessário codificar em 'ISO 8859-1', já o modelo JSON está codificado em UTF16. Portanto, ao ler e consultar os termos do modelo é necessário usar a codificação correta.

In [38]:
# Geração de sentença aleatória que iniciam com n-palavras

# Palavras que serãao usadas como semente
palavra_1 = 'a'
palavra_2 = 'petrobras'

# Ajustando a codificação para ser usado no modelo
palavra_1 = ' ' + palavra_1 + ' '
palavra_2 = ' ' + palavra_2
palavra_1 = palavra_1.encode('utf16')[3:]
palavra_2 = palavra_2.encode('utf16')[4:]
seed = palavra_1 + palavra_2

#Criando sentença aleatória iniciando com as palavras sementes
print(combined_model.make_sentence_with_start(seed.decode()))

 a   p e t r o b r a s   t a m b é m ,    q u e   r e a l i z a   o   m e s m o   p e r f i l   d e   d i s t r i b u i ç ã o   f í s i c a   a   c a m a d a   f í s i c a 


Do modelo criado pela biblioteca Markovify, extraímos a cadeia de Markov e gravamos em um dicionário

In [39]:
# Extrair cadeia de Markov do modelo
chain = combined_model.to_dict()['chain']
# Transformando a cadeia em um dicionário 
dic = {}
for state in json.loads(chain):
    dic[(state[0][0],state[0][1])] = state[1]

Criamos uma função para consultar a cadeia de Markov. Dado duas palavras, extraímos as próximas palavras possíveis, a contagem delas no conjunto de treinamento e a soma total. Dessa forma conseguimos calcular  $p(x_3|x_2,x_1)$.

In [40]:
# Função que dado duas palavras, retorna um dicionários com as probabilidades das palavras seguintes
def query_chain(palavra_1, palavra_2):
    # Selecionar o dicionário cuja entrada é o par de palavras "palavra_1" e "palavra_2"
    d = dic[(palavra_1, palavra_2)]
    new_d = {}
    # Transformar a contagem de palavras em probabilidade
    for w in d:
        new_d[w] = d[w]/sum(d.values())
    return (new_d)

O calculamos a perplexidade para cada sentença. Dada as primeiras palavras, obtivemos do modelo a probabilidade da palavra seguinte. Em seguida, avançamos pela sentença recuperando as probablidades.

In [41]:
# Dada uma sentença, a função retorna uma lista com as entropias calculada para cada trio de palavras
def entropy(sent):
    # Primeiro unimos os MWE
    sent = uni_MWE(sent, mwe)
    # Quebra a sentença em tokens
    words = nltk.word_tokenize(sent)
    # Variável para guardar a entropia de cada conjunto de palavras
    ln_prob_Xi = []
    # Janela de iteração pela sentença
    for w in range(len(words)-2):
        # Cálculo a entropia
        try:
            ln_prob_Xi.append(- math.log(query_chain(words[w], words[w+1])[words[w+2]]))
        except:
            pass

    return(np.mean(ln_prob_Xi))

In [42]:
# Função que recebe uma string com o caminho e nome da pasta com os arquivos e retorna a perplexidade

def perplexity(dirpath):

    # Início do contador de tempo
    start_time = time.time()
    
    # Variável para armazenar as entropias
    ent = []

    # Contagem de arquivos
    n_files = 0
    # Iterando pelos arquivos da pasta indicada em
    for (dirpath, _, filenames) in os.walk(dirpath):
        
        for filename in filenames:
            n_files += 1
            try:
                # Ler o arquivo e calcular a entropia
                with open(os.path.join(dirpath, filename), 'rb') as f:
                    # Preprocessar o texto lido
                    text = preprocess(f.read())
                    #Calcular a entropia da sentença
                    for sent in text:
                        ent.append(entropy(sent))

            except:
                print("Não foi possível ler o arquivo", filename)
        
                
            # Contador de arquivos processados
            if n_files % 5 == 0:
                print ("--- %s arquivos processados ---" % n_files, "--- %s seconds ---" % (time.time() - start_time))
    
    
    # Transformando a lista de entropias em uma np.array e excluir os valores "nan"
    ent = np.array(ent)
    ent = ent[~np.isnan(ent)]
    
    # Fim do contador de tempo
    print ("--- %s seconds ---" % (time.time() - start_time))
    
    return (math.exp(np.mean(ent)))

In [43]:
perplexity("Originais/teste")

C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\upe2\AppData\Roaming\Python\Python36\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--- 5 arquivos processados --- --- 124.25819945335388 seconds ---
--- 10 arquivos processados --- --- 253.72885298728943 seconds ---
--- 354.6159131526947 seconds ---


17.966253778929183